<a href="https://colab.research.google.com/github/phamhoangtin/Analysis-Segmentation-TikiComment/blob/master/ModelAnalysSecmentVietnamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import libraries
import pandas as pd
import json, codecs, os, csv
from time import time
import re
from lxml.etree import tostring
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.preprocessing import sequence
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
df = pd.read_csv("DataClean1.csv",sep =',')

In [0]:
from gensim.models import Word2Vec
input_gensim = []
comments = df.comment_clear.tolist()
for comment in comments:
    input_gensim.append(comment.split())
    
model = Word2Vec(input_gensim, size=128, window=5, min_count=0, workers=4, sg=1)
model.wv.save("word.model")

In [0]:
import gensim.models.keyedvectors as word2vec
model_embedding = word2vec.KeyedVectors.load('./word.model')
word_labels = []
max_seq = 20
embedding_size = 128

for word in model_embedding.vocab.keys():
    word_labels.append(word)
    
def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    lencmt = len(words)

    for i in range(max_seq):
        indexword = i % lencmt
        if (max_seq - i < lencmt):
            break
        if(words[indexword] in word_labels):
            matrix[i] = model_embedding[words[indexword]]
    matrix = np.array(matrix)
    return matrix

In [0]:
df_train = df.sample(10000)
train_data = []
label_data = []
train_data_raw = df_train.comment_clear.to_numpy()
label_data_raw = df_train.label.to_numpy()

for x in train_data_raw:
    train_data.append(comment_embedding(x))
train_data = np.array(train_data)

for y in label_data_raw:
    label_ = np.zeros(2)
    try:
        label_[int(y)] = 1
    except:
        label_[0] = 1
    label_data.append(label_)

In [0]:
sequence_length = 20
embedding_size = 128
num_classes = 2
filter_sizes = 3
num_filters = 150
epochs = 50
batch_size = 30
learning_rate = 0.01
dropout_rate = 0.5
x_train = train_data.reshape(train_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_train = np.array(label_data)


In [0]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(18, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
# Train model
adam = tf.train.AdamOptimizer()
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())


In [0]:
y_train = y_train.astype('float32')
y_train.dtype

In [0]:
model.fit(x = x_train[:300000], y = y_train[:300000], batch_size = batch_size, verbose=1, epochs=epochs, validation_data=(x_train[:30000], y_train[:30000]))

model.save('models.h5')

In [0]:
text = 'hài lòng'
maxtrix_embedding = np.expand_dims(comment_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
print("Label predict: ", result)

In [0]:
text = 'chất lượng kém'
maxtrix_embedding = np.expand_dims(comment_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
print("Label predict: ", result)

In [0]:
text = 'giao hàng nhanh'
maxtrix_embedding = np.expand_dims(comment_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
print("Label predict: ", result)